In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Bluechip/colab models/

/content/drive/MyDrive/Bluechip/colab models


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [ ]:
foods = pd.read_csv('food.csv')
ratings = pd.read_csv('ratings.csv')

In [ ]:
foods.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin..."
1,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni..."
2,3,sweet chilli almonds,Snack,veg,"almonds whole, egg white, curry leaves, salt, ..."
3,4,tricolour salad,Healthy Food,veg,"vinegar, honey/sugar, soy sauce, salt, garlic ..."
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest..."


In [ ]:
ratings.head()

,User_ID,Food_ID,Rating
0,1.0,88.0,4.0
1,1.0,46.0,3.0
2,1.0,24.0,5.0
3,1.0,25.0,4.0
4,2.0,49.0,1.0


In [ ]:
foods.drop(['Describe', 'Veg_Non'], axis=1, inplace=True)

In [ ]:
foods.head()

,Food_ID,Name,C_Type
0,1,summer squash salad,Healthy Food
1,2,chicken minced salad,Healthy Food
2,3,sweet chilli almonds,Snack
3,4,tricolour salad,Healthy Food
4,5,christmas cake,Dessert


In [ ]:
foods_ratings = pd.merge(foods, ratings, on='Food_ID')

In [ ]:
foods_ratings.head()

,Food_ID,Name,C_Type,User_ID,Rating
0,1,summer squash salad,Healthy Food,49.0,5.0
1,1,summer squash salad,Healthy Food,71.0,10.0
2,2,chicken minced salad,Healthy Food,9.0,3.0
3,2,chicken minced salad,Healthy Food,22.0,5.0
4,2,chicken minced salad,Healthy Food,39.0,10.0


In [ ]:
foods_ratings = foods_ratings.groupby(['User_ID', 'Food_ID'], as_index=False).mean()

<ipython-input-193-c02bec5d8d61>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  foods_ratings = foods_ratings.groupby(['User_ID', 'Food_ID'], as_index=False).mean()


In [ ]:
train, test = train_test_split(foods_ratings, test_size=0.2, random_state=42)

In [ ]:
user_food_matrix = train.pivot(index='User_ID', columns='Food_ID', values='Rating').fillna(0)

In [ ]:
item_similarity_matrix = cosine_similarity(user_food_matrix.T)

In [ ]:
item_similarity_data = pd.DataFrame(item_similarity_matrix, index=user_food_matrix.columns, columns=user_food_matrix.columns)

In [ ]:
item_popularity = train.groupby('Food_ID')['Rating'].count().reset_index()

In [ ]:
item_popularity.columns = ['Food_ID', 'Popularity']

In [ ]:
item_popularity = item_popularity.set_index('Food_ID')

In [ ]:
def hybrid_recommendations(user_id, food_id, alpha=0.5):
    similar_items = item_similarity_data[food_id]
    collab_recommendations = user_food_matrix.dot(similar_items)

    item_popularity_scores = item_popularity['Popularity']
    content_recommendations = user_food_matrix.dot(item_popularity_scores)

    hybrid_recommendations = alpha * collab_recommendations + (1 - alpha) * content_recommendations

    recommended_food_ids = hybrid_recommendations.sort_values(ascending=False).index
    recommended_food_names = foods[foods['Food_ID'].isin(recommended_food_ids)]['Name'].tolist()


    return recommended_food_names

In [ ]:
user_id = 20
food_id = 10
alpha_value = 0.7

recommendations = hybrid_recommendations(user_id, food_id, alpha=alpha_value)
print(f"Recommendations for User with User ID {user_id} based on Food ID {food_id}:\n")

for food in recommendations[:10]:
  print(food)

Recommendations for User with User ID 20 based on Food ID 10:

summer squash salad
chicken minced salad
sweet chilli almonds
tricolour salad
christmas cake
japanese curry arancini with barley salsa
chocolate nero cookies
lamb and chargrilled bell pepper soup
cream of almond soup
broccoli and almond soup
